In [ ]:
import os
import torch
import json
import random
from src.Utils import plot_data, create_mask, plot_mask, compare_folders
from src.CustomDataset import CustomDataset
from models import UNet
from torch.utils.data import DataLoader
from torchvision import transforms
from loss import dice_loss

%load_ext autoreload
%autoreload 2

In [ ]:
val_dir = "data/valid"
with open ("data/valid/_annotations.coco.json", "r") as f:
    annotations = json.load(f)
all_images = [os.path.join(val_dir, f) for f in os.listdir(val_dir) if f.endswith(".jpg")]


In [ ]:
random_image_files = random.sample(all_images,2)
plot_data(random_image_files, annotations)


In [ ]:
for part in (os.listdir("data")):
    original_image_dir = f"data/{part}"
    json_file = f"data/{part}/_annotations.coco.json"
    mask_dir = f"data1/{part}/masks"
    img_dir = f"data1/{part}/images"
    create_mask(json_file,mask_dir,img_dir,original_image_dir)

In [ ]:
plot_mask("data1/valid/masks", "data1/valid/images")

In [ ]:
compare_folders("data1/valid/masks", "data1/valid/images")
compare_folders("data1/test/masks", "data1/test/images")
compare_folders("data1/train/masks", "data1/train/images")

In [ ]:
train_path = "data1/train"
test_path = "data1/test"
val_path = "data1/valid"

image_transform = transforms.Compose([
    transforms.Resize(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485], std=[0.229]),  # Assuming grayscale images
    transforms.Lambda(lambda x: x.clamp(0, 1))
])

train_dataset = CustomDataset(train_path, transform=image_transform)
valid_dataset = CustomDataset(val_path, transform=image_transform)
test_dataset = CustomDataset(test_path, transform=image_transform)

train_loader = DataLoader(train_dataset, batch_size=16, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=16, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=16, shuffle=False)

In [ ]:
x, y = next(iter(train_loader))
x.shape , y.shape , type(x) , type(y)

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
model = UNet.UNet().to(device)
loss_fn = dice_loss.DiceLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
epoches = 20

In [ ]:
epochs = 10
for epoch in range(epochs):
    model.train()  # Set model to training mode
    running_loss = 0.0
    correct_predictions = 0
    total_samples = 0
    
    # Iterate over the first 100 data points
    for i, (img, label) in enumerate(train_loader):

        
        img, label = img.to(device), label.to(device)  # Move data to GPU
        optimizer.zero_grad()
        outputs = model(img)
        loss = loss_fn(outputs, label)
        loss.backward()
        optimizer.step()

        _, predicted = torch.max(outputs.data, 1)
        correct_predictions += (predicted == label).sum().item()
        total_samples += label.size(0)
        running_loss += loss.item()
        
        if (i + 1) % 100 == 0:  # print every 100 mini-batches 
            print(f'[{epoch + 1}, {i + 1}] loss: {running_loss / 100:.3f}')
            running_loss = 0.0
    
    # Calculate accuracy after processing all batches for the epoch
    accuracy = correct_predictions / total_samples
    print(f'Accuracy after epoch {epoch + 1}: {accuracy * 100:.2f}%')

print('Finished Training')